In [ ]:
import logging

import numpy as np
import matplotlib.pyplot as plt

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-40s :: %(message)s'
)


# Inverse Obstacle problem corrupted by impulsive noise

Operator that maps the shape of a sound-soft obstacle to the far-field measurements. 
The scattering problem is described by

$$
        \begin{cases}
            \Delta u +\kappa^2 u = 0 & \text{ in } \mathbb{R}^2\backslash\overline{D}\\
             u = 0  & \text{ on } \partial D\\
            \displaystyle{\lim_{r\to\infty}}r^{\frac{1}{2}}(\frac{\partial u^s}{\partial r}-i\kappa u^s)=0 & \text{ for } r=|x|,
        \end{cases}
$$
where $u=u^s+u^i$ is the total field and $D$ is a bounded obstacle in $\mathbb{R}^2$ with $\partial D\in\mathcal{C}^2$. 

Here we assume that only the imaginary part of the far field pattern of $u^s$ can be measured, which is relevant in passive imaging.
    

In [ ]:
from  dirichlet_op import DirichletOp
from regpy.operators import ImaginaryPart

#Forward operator
op0 = DirichletOp(
    kappa = 3,
    N_inc=4
)
op = ImaginaryPart(op0.codomain) * op0


## generate synthetic data with impulsive noise

In [ ]:
from regpy.solvers import RegularizationSetting
from regpy.hilbert import L2, Sobolev
from dirichlet_op import create_synthetic_data
from regpy.vecsps.curve import apple

N_ieq_synth=64
#Exact data
farfield, exact_solution = create_synthetic_data(op0, true_curve=apple(N_ieq_synth,der=3)')
im_farfield = farfield.imag

#Impulsive noise
data = im_farfield.copy()
n,m = data.shape
for i in range(m):
    for j in range(5):
        k = np.random.randint(len(farfield))
        data[k,i] = np.random.randint(10)-5
        #data[k,i] = complex(np.random.randint(10)-5,np.random.randint(10)-5)

#Initial guess
t = 2*np.pi*np.arange(0, op0.N_FK)/op0.N_FK
init = 0.45*np.append(np.cos(t), np.sin(t)).reshape((2, op0.N_FK))
init=init.flatten()

ax = plt.plot(op.codomain.coords[0],data[:,2],label='Im u_{\infty}')

## Standard approach
First we try what happens if we apply a standard IRGNM solver.

In [ ]:
from regpy.solvers.nonlinear.irgnm import IrgnmCG
from regpy.solvers.nonlinear.newton import NewtonCG
import regpy.stoprules as rules

L2setting = RegularizationSetting(op=op, penalty=Sobolev, data_fid=L2)

#Solver: NewtonCG or IrgnmCG
solver = IrgnmCG(L2setting, data, regpar=10, init = init)

stoprule = (
    rules.CountIterations(10) 
)

#Plot function
fig, axs = plt.subplots(1, 2)
axs[0].set_title('Obstacle')
axs[1].set_title('Farfield (real part)')

reco, reco_data = solver.run(stoprule)
axs[0].plot(*exact_solution.z)
axs[0].plot(*op.domain.bd_eval(reco, nvals=op0.N_ieq, nderivs=3).z)

axs[1].plot(op.codomain.coords[0][:,0], im_farfield[:,0], label='exact')
axs[1].plot(op.codomain.coords[0][:,0], reco_data[:,0], label='reco')
axs[1].plot(op.codomain.coords[0][:,0], data[:,0], label='measured')
axs[1].legend()
plt.show()

## Reconstruction with Huber data fidelity term

Now introduce a Huber data fidelity term $H_{\sigma}$ ...

In [ ]:
from regpy.solvers import TikhonovRegularizationSetting
from regpy.solvers.linear.semismoothNewton import SemismoothNewton_bilateral
from regpy.functionals import Huber,HorizontalShiftDilation

alpha = 10.
sigma = 0.1
Sdat = (1./sigma)*Huber(op.codomain,sigma=sigma)

fk = init

... and iteratively solve 
$$
    f^{l+1}  \in \mathrm{argmin}_f \left[\frac{1}{\alpha_l}H_{\sigma}(F'[f^l](f-f^l) + F(f^l) -g^{\mathrm{obs}}+ \|f-f_0\|^2\right]  
$$

Run the following cell many times to see the reconstructions improving!

In [ ]:
yk,deriv = op.linearize(fk)
huber_setting = TikhonovRegularizationSetting(deriv,Sobolev,Sdat,alpha,data_fid_shift=data-yk)
dual_setting = huber_setting.dualSetting()
solver = SemismoothNewton_bilateral(dual_setting)
_,Tstar_p = solver.run(stoprule=rules.CountIterations(50))
update = huber_setting.dualToPrimal(Tstar_p,argumentIsOperatorImage=True)
fk+= update
alpha *=0.5

plt.plot(*exact_solution.z)
plt.plot(*op.domain.bd_eval(fk, nvals=op0.N_ieq, nderivs=3).z)